# Random Data Simulation and Fitting Isotopomer Distribution Using Neural Network

Addtion of neural network to fit isotopomer distribution. We will complete this with the following workflow: 
- Create simple nn to take the place of the basic fit within the isotopomer class.
- Expand this to handle multiple samples, adding functions to generate distributions and sim data for different samples.
- Train and tune the network and add overfitting prevention measures.
- Generalise the network to handle different metabolites.
- Use our networks to fit to real HSQC and GCMS data

Import necessary packages:

In [ ]:
import numpy as np
import pandas as pd
from metabolabpytools import isotopomerAnalysis

Create an isotopomerAnalysis object:

In [ ]:
ia = isotopomerAnalysis.IsotopomerAnalysis()

Define metabolite parameters:

In [ ]:
# Ensure isotopomers is correctly initialized
isotopomers = [
    [0, 0, 0],  # Unlabelled
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1]
]

num_samples = 1000
hsqc = [0, 1, 1]
metabolite = 'L-LacticAcid'


In [ ]:
ia.init_metabolite_multiple_samples(metabolite, hsqc, num_samples=num_samples)

Initialising and set isoptomer, HSQC and gcms data for multiple samples:

In [ ]:
generated_percentages = []
for exp_index in range(num_samples):
    random_percentages = ia.generate_isotopomer_percentages()  # Generate new random percentages for each sample
    generated_percentages.append(random_percentages)  # Store generated percentages for comparison
    
    ia.set_fit_isotopomers_simple(metabolite=metabolite, isotopomers=isotopomers, percentages=random_percentages, exp_index=exp_index)
    ia.sim_hsqc_data(metabolite=metabolite, exp_index=exp_index, isotopomers=isotopomers, percentages=random_percentages)
    ia.sim_gcms_data(metabolite, exp_index)

Add noise to HSQC and GC-MS data:

In [ ]:
ia.add_noise_to_hsqc_gcms(metabolite, num_samples, hsqc_noise_level=0.03, gcms_noise_level=0.075)

Modify object states for the data:

In [ ]:
ia.use_hsqc_multiplet_data = True
ia.use_gcms_data = True
ia.use_nmr1d_data = False

Fitting the neural network:

In [ ]:
ia.fit_data_nn(metabolite=metabolite, fit_isotopomers=isotopomers, percentages=generated_percentages, num_samples=num_samples)

## Addressing Overfitting: 

To prevent overfitting in my neural network model for predicting isotopomer distributions, several strategies have been implemented:

- First, use of a validation set to monitor the model's performance during training, ensuring it maintains its ability to generalize to unseen data has been used. This involves splitting the data into training and validation sets and using early stopping to halt training when the validation loss stops improving, which helps avoid overfitting by preventing the model from learning noise in the training data. 
 
- Additionally, dropout layers have been employed within the neural network architecture. Dropout randomly deactivates a fraction of neurons during each training step, which forces the network to learn more robust features and reduces reliance on any specific neurons. 

- Regularization techniques, such as L2 regularization, have been used to penalize large weights, discouraging the model from becoming too complex. 

- Finally, the model has been trained with an adequate amount of data (1000 samples), enhancing the model's ability to generalize.

## Generalising for other metabolites:

In [2]:
import numpy as np
import pandas as pd
from metabolabpytools import isotopomerAnalysis

ia = isotopomerAnalysis.IsotopomerAnalysis()

# Define isotopomers for different metabolites
isotopomers_three_carbon = [
    [0, 0, 0],  # Unlabelled
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 1]
]

isotopomers_aspartate = [
    [0, 0, 0, 0],  # Unlabelled
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 1],
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 1],
    [0, 1, 1, 1],
    [1, 1, 1, 1]
]

num_samples = 1000
hsqc_three_carbon = [[1, 1, 1], [0, 1, 1]]
hsqc_aspartate = [
    [0, 1, 1, 0], 
    [1, 1, 1, 0], 
    [0, 1, 1, 1], 
    [1, 1, 1, 1]
]

# Initialize the metabolites with multiple samples
ia.init_metabolite_multiple_samples(metabolites=['three-carbon', 'aspartate'], hsqc=hsqc_three_carbon, num_samples=num_samples)

# Generate and set isotopomers for three-carbon metabolite
for hsqc in hsqc_three_carbon:
    generated_percentages = []
    for exp_index in range(num_samples):
        random_percentages = ia.generate_isotopomer_percentages('three-carbon')
        generated_percentages.append(random_percentages)
        
        ia.set_fit_isotopomers_simple(metabolite='three-carbon', isotopomers=isotopomers_three_carbon, percentages=random_percentages, exp_index=exp_index)
        ia.sim_hsqc_data(metabolite='three-carbon', exp_index=exp_index, isotopomers=isotopomers_three_carbon, percentages=random_percentages)
        ia.sim_gcms_data('three-carbon', exp_index)

    ia.add_noise_to_hsqc_gcms('three-carbon', num_samples, hsqc_noise_level=0.03, gcms_noise_level=0.075)
    ia.fit_data_nn(metabolite='three-carbon', fit_isotopomers=isotopomers_three_carbon, percentages=generated_percentages, num_samples=num_samples, hsqc=hsqc, tuner_project_name=f'three_carbon_{hsqc}')

# Generate and set isotopomers for aspartate
for hsqc in hsqc_aspartate:
    generated_percentages = []
    for exp_index in range(num_samples):
        random_percentages = ia.generate_isotopomer_percentages('aspartate')
        generated_percentages.append(random_percentages)
        
        ia.set_fit_isotopomers_simple(metabolite='aspartate', isotopomers=isotopomers_aspartate, percentages=random_percentages, exp_index=exp_index)
        ia.sim_hsqc_data(metabolite='aspartate', exp_index=exp_index, isotopomers=isotopomers_aspartate, percentages=random_percentages)
        ia.sim_gcms_data('aspartate', exp_index)

    ia.add_noise_to_hsqc_gcms('aspartate', num_samples, hsqc_noise_level=0.03, gcms_noise_level=0.075)
    ia.fit_data_nn(metabolite='aspartate', fit_isotopomers=isotopomers_aspartate, percentages=generated_percentages, num_samples=num_samples, hsqc=hsqc, tuner_project_name=f'aspartate_{hsqc}')

# Save results to an Excel file
ia.save_results('results.xlsx')


Trial 200 Complete [00h 01m 47s]
val_loss: 20.53839683532715

Best val_loss So Far: 17.318084716796875
Total elapsed time: 02h 34m 27s
Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 276.7698 - val_loss: 270.1278
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 259.3806 - val_loss: 261.0565
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 249.4652 - val_loss: 249.9429
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 241.4877 - val_loss: 245.6904
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 238.3989 - val_loss: 243.3895
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 236.5983 - val_loss: 241.0748
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 242.3429 - val_loss: 239.4780
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 234.2017 - val_loss: 238.5572
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 242.2497 - val_loss: 237.9764
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 232